In [1]:
import pandas as pd

df21 = pd.read_csv("/opt/conda/anaconda/lib/python3.6/site-packages/irsx/CSV/index_2021.csv",index_col=False)
object_ids = df21["OBJECT_ID"].to_numpy()

/opt/conda/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
object_ids_s = object_ids[:100]
object_ids_s

array([202010659349301301, 202010729349300041, 202010729349300021,
       202010729349300836, 202000669349300135, 202000669349301125,
       202000669349301320, 202000669349301850, 202000669349300905,
       202000669349300700, 202000669349300425, 202000679349300505,
       202000679349300405, 202000679349300210, 202020669349300127,
       202000729349300710, 202020689349300402, 202030699349301268,
       202000699349300330, 202000699349300740, 202010679349300301,
       202020699349301072, 202020699349300617, 202030709349300313,
       202000709349301455, 202000709349300200, 202000709349300520,
       202010699349301266, 202010699349300541, 202000719349301035,
       202010709349300601, 202010709349301231, 202010709349300931,
       202010709349301506, 202020719349300727, 202010719349300816,
       202010719349301566, 202010719349301446, 202040699349300954,
       202040669349300604, 202040709349301429, 202040719349301204,
       202020589349300437, 202020589349300007, 202020589349301

In [3]:
from irsx.xmlrunner import XMLRunner
xml_runner = XMLRunner()
list_all = []
for e in object_ids_s:
    filing = xml_runner.run_filing(e)
    schedules = filing.list_schedules()
    
    ein = 0
    state = 0
    name = 0
    revenue = 0
    revenueEZ = 0
    
    if "ReturnHeader990x" in schedules:
        header = filing.get_parsed_sked("ReturnHeader990x")
        #print(len(header))
        header_part_i = header[0]["schedule_parts"]["returnheader990x_part_i"]
        #print(header_part_i)
        ein = header_part_i["ein"]
        try:
            state = header_part_i["USAddrss_SttAbbrvtnCd"]
        except KeyError:
            state = XX
        name = header_part_i["BsnssNm_BsnssNmLn1Txt"]
        #print(ein, state, name)
        
    if "IRS990EZ" in schedules:
        irs990ez = filing.get_parsed_sked("IRS990EZ")
        irs990ez_part_i = irs990ez[0]["schedule_parts"]["ez_part_i"]
        revenueEZ = irs990ez_part_i["TtlRvnAmt"]        
    
    if "IRS990" in schedules:
        irs990 = filing.get_parsed_sked("IRS990")
        #print(len(irs990))
        irs990_part_i = irs990[0]["schedule_parts"]["part_i"]
        revenue = irs990_part_i["CYTtlRvnAmt"]
        #print(revenue)
        
    list = [e, ein, state, name, revenue, revenueEZ]
    list_all.append(list)
    
df_result = pd.DataFrame(list_all, columns =['ObjectID', 'EIN', 'State', 'Name', 'Revenue', 'RevenueEZ'])
df_result.to_csv('revenue_2021_100.csv', index = False)
df_result
  

        
    #print(e, ein, state, name, revenue, revenueEZ)

,ObjectID,EIN,State,Name,Revenue,RevenueEZ
0,202010659349301301,452772761,NJ,CAMDEN'S CHARTER SCHOOL,5148336,0
1,202010729349300041,237061115,MS,JACKSON STATE UNIVERSITY,10386900,0
2,202010729349300021,344427516,OH,TIFFIN UNIVERSITY,64789812,0
3,202010729349300836,840865247,CO,NETWORK MINISTRIES INC,218029,0
4,202000669349300135,205647589,SC,FAMILY PROMISE OF BEAUFORT COUNTY,482668,0
...,...,...,...,...,...,...
95,202021379349100307,412209842,GA,LOVE PEACE AND PROSPERITY INTERNATIONAL INC,0,0
96,202021379349100602,136216916,FL,SIMON AND STELLA SHEIB FOUNDATION,0,0
97,202021379349200022,202272015,SC,ALPHA SIGMA PHI FRATERNITY INC,0,198372
98,202021379349200512,431336280,MO,MISSOURI DAIRY ASSOCIATIONINC,0,94662


In [4]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext(appName="BigDataIRS")
spark = SparkSession(sc)

In [5]:
#csv_path = "gs://sparkbucket02/revenue_2021_100.csv"
#gs://movie_files_101/u.item


# so wie ich das verstanden habe wird nun auf das lokale File zugegriffen ..
# sprich wenn wir mehrere Worker Nodes haben, müssen alle Worker auch dieses File haben => funktioniert wohl, aber HDFS benutzen wäre wohl cooler
# https://stackoverflow.com/questions/42091575/pyspark-load-file-path-does-not-exist
csv_path = "file:///spark_csv/revenue_2021_100.csv"
csv_df = spark.read.csv(csv_path, header=True)
csv_df.head(100)

[Row(ObjectID='202010659349301301', EIN='452772761', State='NJ', Name="CAMDEN'S CHARTER SCHOOL", Revenue='5148336', RevenueEZ='0'),
 Row(ObjectID='202010729349300041', EIN='237061115', State='MS', Name='JACKSON STATE UNIVERSITY', Revenue='10386900', RevenueEZ='0'),
 Row(ObjectID='202010729349300021', EIN='344427516', State='OH', Name='TIFFIN UNIVERSITY', Revenue='64789812', RevenueEZ='0'),
 Row(ObjectID='202010729349300836', EIN='840865247', State='CO', Name='NETWORK MINISTRIES INC', Revenue='218029', RevenueEZ='0'),
 Row(ObjectID='202000669349300135', EIN='205647589', State='SC', Name='FAMILY PROMISE OF BEAUFORT COUNTY', Revenue='482668', RevenueEZ='0'),
 Row(ObjectID='202000669349301125', EIN='311131742', State='CA', Name='Pacific Local Masters Swimming Committee', Revenue='143394', RevenueEZ='0'),
 Row(ObjectID='202000669349301320', EIN='271969337', State='OR', Name='ST MARYS OUTREACH INC', Revenue='105375', RevenueEZ='0'),
 Row(ObjectID='202000669349301850', EIN='474824100', State=

In [33]:
#hdfs_csv_path = "hdfs://spark-jupyter-m/spark_csv/revenue_2021_100.csv"
#hdfs_csv_path = "hdfs:///spark_csv/revenue_2021_100.csv"
#hdfs_csv_path = "spark_csv/revenue_2021_100.csv"
hdfs_csv_path = "hdfs://spark-jupyter-m/user/hdfs/spark_csv/revenue_2021_100.csv"
hdfs_df = spark.read.csv(hdfs_csv_path, header = True, inferSchema = True)
#hdfs_df.parallelize()
#hdfs_df.head(100)
#hdfs_df.collect()
hdfs_df.printSchema()

hdfs_drop = hdfs_df.drop('ObjectID','EIN','Name','RevenueEZ')
#hdfs_drop.printSchema()

hdfs_rdd = hdfs_drop.rdd
#hdfs_rdd.collect()

from operator import add
reduced_rdd = hdfs_rdd.reduceByKey(add).sortBy(lambda x: x[1], ascending = False)
reduced_rdd.collect()


#sparkTuple = hdfs_rdd.map(lambda letter: (RA(letter), letter))

#rdd.flatMap(lambda x: [(x, x), (x, x)]).collect())

root
 |-- ObjectID: long (nullable = true)
 |-- EIN: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Revenue: integer (nullable = true)
 |-- RevenueEZ: integer (nullable = true)



[('PA', 321670744),
 ('OH', 65537128),
 ('MS', 11226629),
 ('MA', 11116540),
 ('TN', 8394292),
 ('FL', 6715765),
 ('NJ', 5925789),
 ('CA', 1661182),
 ('TX', 1658973),
 ('AL', 1592993),
 ('WA', 1498439),
 ('NY', 1454146),
 ('NE', 1304951),
 ('KY', 862110),
 ('SC', 799249),
 ('NC', 716868),
 ('MD', 648602),
 ('IL', 536840),
 ('SD', 517835),
 ('MI', 421865),
 ('CT', 376019),
 ('OK', 322235),
 ('CO', 293081),
 ('KS', 292627),
 ('UT', 273713),
 ('VA', 256644),
 ('VT', 172737),
 ('ME', 111116),
 ('OR', 105375),
 ('IA', 88462),
 ('WV', 61224),
 ('GA', 47408),
 ('AK', 41841),
 ('LA', 31847),
 ('IN', 16021),
 ('WI', 0),
 ('NM', 0),
 ('MO', 0)]

In [7]:
#from pyspark import SparkContext
#from pyspark.sql import SQLContext
#from pyspark import SparkFiles
#sc = SparkContext(appName="BigDataIRS")
#sc.setLogLevel("ERROR")
#sqlContext = SQLContext(sc)
#df = sqlContext.read.csv("hdfs:///spark_csv/revenue_2021_100.csv", header=True, inferSchema= True)
#df = sqlContext.read.csv("hdfs:///spark_csv/revenue_2021_100.csv", header=True, inferSchema= True)
#df.head()

In [1]:
#from google.cloud import storage
#client = storage.Client()
# https://console.cloud.google.com/storage/browser/[bucket-id]/
#bucket = client.get_bucket('sparkbucket02')
# Then do other things...
#blob = bucket.get_blob("revenue_2021_100.csv") #('remote/path/to/file.txt')

#df = pd.read_csv(blob.download_as_string())
#df

In [2]:
#spark = SparkSession \
#    .builder \
#    .appName("Protob Conversion to Parquet") \
#    .config("spark.some.config.option", "some-value") \
#    .getOrCreate()\

#df = spark.read.csv('/home/hadoop/observations_temp.csv, header=True)

sudo su - hdfs
hdfs dfsadmin -safemode leave

hdfs dfs -mkdir spark_csv
hdfs dfs -put /spark_csv/revenue_2021_100.csv spark_csv/revenue_2021_100.csv
hdfs dfs -ls spark_csv

https://stackoverflow.com/questions/42091575/pyspark-load-file-path-does-not-exist
https://stackoverflow.com/questions/33055403/how-to-navigate-directories-in-hadoop-hdfs
https://stackoverflow.com/questions/28213116/hadoop-copy-a-local-file-system-folder-to-hdfs
https://stackoverflow.com/questions/61197811/can-i-read-csv-files-from-google-storage-using-spark-in-more-than-one-executor
https://groups.google.com/g/cloud-dataproc-discuss/c/cubkWrjkk2g?pli=1
https://stackoverflow.com/questions/56448009/storing-source-file-in-google-dataproc-hdfs-vs-google-cloud-storagegoogle-bucke

hdfs dfs -put /spark_csv/revenue_2021_100.csv /user/root/spark_csv/revenue_2021_100.csv